# Setup

In [1]:
%%bash
if [[ -d project-tycho-utilities ]];
then
  cd project-tycho-utilities/
  git pull
else
  git clone https://github.com/lgautier/project-tycho-utilities.git
  cd project-tycho-utilities/
fi
DBNAME=../tycho.db make all

wget -q --show-progress http://www.healthdata.gov/sites/default/files/ProjectTycho_Level2_v1.1.0_0.zip
unzip ProjectTycho_Level2_v1.1.0_0.zip && rm ProjectTycho_Level2_v1.1.0_0.zip
Makefile:9: recipe for target 'ProjectTycho_Level2_v1.1.0.csv.gz' failed
rm ProjectTycho_Level2_v1.1.0_0.zip


Cloning into 'project-tycho-utilities'...

     0K .......... .......... .......... .......... ..........  0%  738K 29s
    50K .......... .......... .......... .......... ..........  0%  896K 27s
   100K .......... .......... .......... .......... ..........  0% 3.25M 20s
   150K .......... .......... .......... .......... ..........  0% 1.18M 19s
   200K .......... .......... .......... .......... ..........  1% 3.20M 17s
   250K .......... .......... .......... .......... ..........  1% 18.3M 14s
   300K .......... .......... .......... .......... ..........  1% 20.8M 12s
   350K .......... .......... .......... .......... ..........  1% 1.34M 13s
   400K .......... .......... .......... .......... ..........  2% 6.46M 12s
   450K .......... .......... .......... .......... ..........  2% 6.04M 11s
   500K .......... .......... .......... .......... ..........  2% 28.4M 10s
   550K .......... .......... .......... .......... ..........  2% 26.5M 9s
   600K .......... .......... ....

<!-- label:spark_setup -->
Spark can be started from regular Python code.

In [2]:
import findspark
findspark.init()

import pyspark

conf = pyspark.conf.SparkConf()
(conf.setMaster('local[2]')
 .setAppName('ipython-notebook')
 .set("spark.executor.memory", "2g"))

sc = pyspark.SparkContext(conf=conf)

ModuleNotFoundError: No module named 'findspark'

<!-- label:spark_sql_create -->

In [ ]:
import sqlite3
dbfilename = "tycho.db"
dbcon = sqlite3.connect(dbfilename)
cursor = dbcon.cursor()

<!-- label:spark_dataframe -->

In [ ]:
from pyspark.sql import SQLContext, Row
sqlcontext = SQLContext(sc)
cursor.execute("SELECT * FROM location")
location = \
    sqlcontext.createDataFrame(cursor,
                               tuple(x[0] for x in cursor.description))
location.registerTempTable("location")

sql = """
SELECT * 
FROM (SELECT * FROM disease WHERE name='PNEUMONIA') AS disease
INNER JOIN casecount
ON disease.id=casecount.disease_id"""

cursor.execute(sql)
casecount = \
    sqlcontext.createDataFrame(cursor,
                               tuple(x[0] for x in cursor.description))
casecount.registerTempTable("casecount")

Streaming data out of SQLite onto Spark cluster
SQL query on database in Spark
Regular old SQL
    Translated by Spark
    optimized
    sent to JVM for bytecode

---


<!-- label:spark_query -->
SQL can be used to query the data.

In [ ]:
sql = """
SELECT state, count(city) AS ct
FROM location
GROUP BY state
ORDER BY ct DESC
"""
counts = sqlcontext.sql(sql)

<!-- label:spark_query_collect -->
The evaluation is only performed when the results are needed.

In [ ]:
res = counts.collect()
res[:3]

<!-- label:spark_mapreduce -->

Spark is particularly comfortable with map-reduce tasks.
The input data can be our table (stored in a RDBM).
Here we count the number of times suffixes are found in city names:

In [ ]:
names = (location
         .filter(location.city.isNotNull())
         .rdd
         .flatMap(lambda rec: [x[-5:] for x in rec.city.split()])
         .map(lambda word: (word.lower(), 1))
         .reduceByKey(lambda a, b: a+b))
names.takeOrdered(10, key = lambda x: -x[1])

---

<!-- label:spark_sqlmapreduce -->

We can also seamlessly use result table obtained from an SQL query
to perform map/reduce tasks:

In [ ]:
sql = """
SELECT city
FROM (SELECT * FROM casecount WHERE epiweek LIKE '1912%') AS sub
INNER JOIN location
ON location.id=sub.location_id
GROUP BY city
"""
y_1912 = sqlcontext.sql(sql)
names = (y_1912
         .filter(y_1912.city.isNotNull())
         .rdd
         .flatMap(lambda rec: [x[-5:] for x in rec.city.split()])
         .map(lambda word: (word.lower(), 1))
         .reduceByKey(lambda a,b: a+b))
names.takeOrdered(5, key = lambda x: -x[1])

---

<!-- label:spark_sqlmapreduceggplot_1_2 -->

# SQL query

In [ ]:
## --- SQL ---
sql = """
SELECT state, city, date_from, event, count AS ct
FROM (SELECT * FROM casecount WHERE epiweek LIKE '1912%') AS sub
INNER JOIN location
ON location.id=sub.location_id
"""

y_1912 = sqlcontext.sql(sql)

## --- Spark ---
cases = (y_1912
         .rdd
         .map(lambda rec: ((rec.state,
	                    int(rec.date_from.split('-')[1]),
			    rec.event),
                           rec.ct))
         .reduceByKey(lambda a, b: a + b)).collect()